In [1]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import numpy as np
import xgboost as xgb

xgb_dir = "/home/yannick/Work/3p/xgboost/demo"

### load data in do training
dtrain = xgb.DMatrix(xgb_dir + '/data/agaricus.txt.train')


/home/yannick/bin/anaconda3/envs/py27/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [10]:
float(r.tail(1)["test-error-mean"])

0.0012283333333333332

In [19]:
num_round = 250

def hyperopt_train_test(params):

    params["objective"] = 'binary:logistic'
    params["silent"] = 0
    params['max_depth'] = 1+ params["max_depth"]
    
    print "=>", params, 
    
    r = xgb.cv(params, dtrain, num_round, nfold=5,
               metrics={'error'})
   
    weight = float(r.tail(1)["test-error-mean"])
    print "=>>", weight
    
    return weight

space4xgb = {
    'max_depth': hp.choice('max_depth', range(1, 3)),
    'eta':       hp.loguniform('eta', 0.001, 0.5),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.2, 1.0)
}

def f(params):
    acc = hyperopt_train_test(params)
    return {'loss': acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, space4xgb, algo=tpe.suggest, max_evals=20, trials=trials)
print 'best:'
print best
print hyperopt_train_test(best)

=> {'objective': 'binary:logistic', 'eta': 1.0460172721556036, 'colsample_bytree': 0.9200286144832712, 'max_depth': 2, 'silent': 0} =>> 0.0
=> {'objective': 'binary:logistic', 'eta': 1.22617458308389, 'colsample_bytree': 0.8086381300819385, 'max_depth': 2, 'silent': 0} =>> 0.0
=> {'objective': 'binary:logistic', 'eta': 1.4177914763865918, 'colsample_bytree': 0.9093171045972901, 'max_depth': 2, 'silent': 0} =>> 0.0
=> {'objective': 'binary:logistic', 'eta': 1.3263954749232159, 'colsample_bytree': 0.5006052528019884, 'max_depth': 2, 'silent': 0} =>> 0.0
=> {'objective': 'binary:logistic', 'eta': 1.4262058194108522, 'colsample_bytree': 0.6576934678086898, 'max_depth': 3, 'silent': 0} =>> 0.0
=> {'objective': 'binary:logistic', 'eta': 1.5895358325207551, 'colsample_bytree': 0.6523287043681503, 'max_depth': 3, 'silent': 0} =>> 0.064209
=> {'objective': 'binary:logistic', 'eta': 1.088150004146515, 'colsample_bytree': 0.99441477872321, 'max_depth': 3, 'silent': 0} =>> 0.0
=> {'objective': 'bi

In [20]:
for t in trials:
    print t['misc']['vals'], '->\t', t['result']['loss']

{'eta': [1.0460172721556036], 'colsample_bytree': [0.9200286144832712], 'max_depth': [0]} ->	0.0
{'eta': [1.22617458308389], 'colsample_bytree': [0.8086381300819385], 'max_depth': [0]} ->	0.0
{'eta': [1.4177914763865918], 'colsample_bytree': [0.9093171045972901], 'max_depth': [0]} ->	0.0
{'eta': [1.3263954749232159], 'colsample_bytree': [0.5006052528019884], 'max_depth': [0]} ->	0.0
{'eta': [1.4262058194108522], 'colsample_bytree': [0.6576934678086898], 'max_depth': [1]} ->	0.0
{'eta': [1.5895358325207551], 'colsample_bytree': [0.6523287043681503], 'max_depth': [1]} ->	0.064209
{'eta': [1.088150004146515], 'colsample_bytree': [0.99441477872321], 'max_depth': [1]} ->	0.0
{'eta': [1.0883569950843255], 'colsample_bytree': [0.7228370828999071], 'max_depth': [1]} ->	0.0
{'eta': [1.4130513233275737], 'colsample_bytree': [0.5911255503619075], 'max_depth': [1]} ->	0.0
{'eta': [1.632121956767438], 'colsample_bytree': [0.28397131533839454], 'max_depth': [1]} ->	0.1588326
{'eta': [1.3938097980479

In [21]:
range(1,20)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [22]:
best

{'colsample_bytree': 0.9200286144832712,
 'eta': 1.0460172721556036,
 'max_depth': 1,
 'objective': 'binary:logistic',
 'silent': 0}

In [23]:
r = xgb.cv(best, dtrain, 200)
r.tail(1)

,test-error-mean,test-error-std,train-error-mean,train-error-std
199,0.0,0.0,0.0,0.0
